In [1]:
import os
import glob
import shutil
import SimpleITK as sitk
import nibabel as nib
import numpy as np
from tqdm.notebook import tqdm
import SimpleITK as sitk
import itk


In [2]:
root_dir = "/scratch/scratch6/akansh12/Parse_data/evaluation/"
test_files_path = sorted(glob.glob(os.path.join(root_dir, "*.nii.gz")))
test_data = [{"images": image_name } for image_name in test_files_path]

In [3]:
def GetLargestConnectedCompont(binarysitk_image):
    """
    save largest object
    :param sitk_maskimg:binary itk image
    :return: largest region binary image
    """
    cc = sitk.ConnectedComponent(binarysitk_image)
    stats = sitk.LabelIntensityStatisticsImageFilter()
    stats.SetGlobalDefaultNumberOfThreads(8)
    stats.Execute(cc, binarysitk_image)
    maxlabel = 0
    maxsize = 0
    for l in stats.GetLabels():
        size = stats.GetPhysicalSize(l)
        if maxsize < size:
            maxlabel = l
            maxsize = size
    labelmaskimage = sitk.GetArrayFromImage(cc)
    outmask = labelmaskimage.copy()
    outmask[labelmaskimage == maxlabel] = 1
    outmask[labelmaskimage != maxlabel] = 0
    outmask_sitk = sitk.GetImageFromArray(outmask)
    outmask_sitk.SetDirection(binarysitk_image.GetDirection())
    outmask_sitk.SetSpacing(binarysitk_image.GetSpacing())
    outmask_sitk.SetOrigin(binarysitk_image.GetOrigin())
    return outmask_sitk


In [4]:
unet_8909_files = sorted(glob.glob("./submit_orig_8927_main_artery/*/*.nii.gz") )
swin_8940_files = sorted(glob.glob("./swin_8940_main_artery/*/*.nii.gz") )
unet_8927_files = sorted(glob.glob("./submit_orig_8909_main_artery/*/*.nii.gz")) 

In [5]:
mask_images = [unet_8909_files,
swin_8940_files,
unet_8927_files]

In [7]:
for num,i in tqdm(enumerate(test_data)):
    input_image = sitk.ReadImage(i['images'])
    input_array = sitk.GetArrayFromImage(input_image)
    
    masked = 0
    for j in tqdm(mask_images):
        masked += nib.load(j[num]).get_fdata()
        
    masked = masked/len(mask_images)
    masked[np.nonzero(masked >= 0.5)] = 1.0
    masked[np.nonzero(masked < 0.5)] = 0.0
    
    mask_image = sitk.GetImageFromArray(np.swapaxes(masked, 0,2))
    masked = sitk.BinaryThreshold(mask_image, lowerThreshold=0, upperThreshold=0.5, insideValue=0,
                       outsideValue=1)
    
    mask_image = GetLargestConnectedCompont(masked)
    mask_image.SetOrigin(input_image.GetOrigin())
    mask_image.SetSpacing(input_image.GetSpacing())
    sitk.WriteImage(mask_image, "./submit/"+i['images'].split('/')[-1])
    

0it [00:00, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
!zip -r submit.zip ./submit/

  adding: submit/ (stored 0%)
  adding: submit/PA000219.nii.gz (deflated 93%)
  adding: submit/PA000164.nii.gz (deflated 94%)
  adding: submit/PA000316.nii.gz (deflated 95%)
  adding: submit/PA000105.nii.gz (deflated 93%)
  adding: submit/PA000032.nii.gz (deflated 94%)
  adding: submit/PA000136.nii.gz (deflated 94%)
  adding: submit/PA000122.nii.gz (deflated 93%)
  adding: submit/PA000013.nii.gz (deflated 93%)
  adding: submit/PA000044.nii.gz (deflated 94%)
  adding: submit/PA000312.nii.gz (deflated 95%)
  adding: submit/PA000114.nii.gz (deflated 94%)
  adding: submit/PA000087.nii.gz (deflated 95%)
  adding: submit/PA000126.nii.gz (deflated 94%)
  adding: submit/PA000059.nii.gz (deflated 93%)
  adding: submit/PA000069.nii.gz (deflated 95%)
  adding: submit/PA000051.nii.gz (deflated 94%)
  adding: submit/PA000218.nii.gz (deflated 94%)
  adding: submit/PA000117.nii.gz (deflated 93%)
  adding: submit/PA000269.nii.gz (deflated 93%)
  adding: submit/PA000172.nii.gz (deflated 94%)
  adding: 